In [11]:
import Text.Read
import Data.Maybe
import Data.Char
import Data.List.Split
import qualified Data.Vector as V
import qualified Data.Map as M

In [12]:
type Reg = Char
data Value = Value Int | Register Reg deriving Show
data Instruction = Sound Value
                 | Set Reg Value
                 | Add Reg Value
                 | Mul Reg Value
                 | Mod Reg Value
                 | Recover Value
                 | Jump Value Value deriving Show

In [13]:
readRegister = head . dropWhile isSpace
readValue s =
  let r = Register $ readRegister s
      v = Value <$> readMaybe s
    in fromMaybe r v

In [14]:
readLine :: String -> Instruction
readLine s = case i of
  "snd" -> Sound (readValue $ args !! 0)
  "set" -> Set (readRegister $ args !! 0)
               (readValue $ args !! 1)
  "add" -> Add (readRegister $ args !! 0)
               (readValue $ args !! 1)
  "mul" -> Mul (readRegister $ args !! 0)
               (readValue $ args !! 1)
  "mod" -> Mod (readRegister $ args !! 0)
               (readValue $ args !! 1)
  "rcv" -> Recover (readValue $ args !! 0)
  "jgz" -> Jump (readValue $ args !! 0)
                (readValue $ args !! 1)
  where i = take 3 s
        args = splitOn " " $ drop 4 s

In [15]:
type Machine = M.Map Reg Int
type Pc = Int
type Instructions = V.Vector Instruction
step :: (Machine, Pc, [Int]) -> Instructions
                             -> (Machine, Pc, [Int])
step (m, pc, rs) ins = case i of
  Sound v -> (setValue ' ' v, pc + 1, rs)
  Set a b -> (setValue a b, pc + 1, rs)
  Add a b -> (setValue a $ Value
                         $ (getValue b) +
                           (getValue $ Register a),
              pc + 1, rs)
  Mul a b -> (setValue a $ Value
                         $ (getValue b) *
                           (getValue $ Register a),
              pc + 1, rs)
  Mod a b -> (setValue a $ Value
                         $ (getValue $ Register a) `mod`
                           (getValue b),
              pc + 1, rs)
  Recover v -> if (getValue v /= 0) then (m, pc + 1, (getValue $ Register ' '):rs)
                                    else (m, pc + 1, rs)
  Jump a b -> if (getValue a > 0) then (m, pc + getValue b, rs)
                                  else (m, pc + 1, rs)
  where i = ins V.! pc
        getValue (Value i) = i
        getValue (Register r) = fromMaybe 0 $ M.lookup r m
        setValue r v = M.insert r (getValue v) m

In [16]:
run :: Instructions -> [(Machine, Pc, [Int])]
run ins = run' (M.empty, 0, [])
  where run' mpc@(_, pc, _) = if (pc < 0 || pc >= V.length ins)
                              then [mpc]
                              else mpc:(run' (step mpc ins))

In [17]:
head . filter (\(a,b,c) -> not $ null c) . run . fmap readLine . V.fromList . lines <$> readFile "input18.txt"

(fromList [(' ',9423),('a',2147483647),('b',9423),('f',0),('i',126),('p',1928979423)],26,[9423])